# Visualize with Bokeh
참고 : https://docs.bokeh.org/en/latest/docs/user_guide.html


In [53]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
#plt.style.use('ggplot')
#plt.style.use('fivethirtyeight')
import scipy as sp

In [54]:
# warning 무시
import warnings
warnings.filterwarnings(action='ignore')

In [55]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [56]:
from math import pi
import pandas as pd
from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.models import LabelSet, ColumnDataSource
from bokeh.models import HoverTool

In [57]:
ActiveUser=pd.DataFrame({'month':[4,5,6,7,8,9,10],
    'noActive': [37042,36246,45647,5884,36311,53051,34414],
             'some':[283525,263560,298522,305502,296193,289340,241853],
              'Active_low':[197346,185709,218458,273477,225868,211300,168484],
              'Active_med':[70285,68162,92722,108733,97921,78510,62436],
              'Active_high':[39779,39690,47182,59787,50470,43843,3121],
             })
ActiveUser

,month,noActive,some,Active_low,Active_med,Active_high
0,4,37042,283525,197346,70285,39779
1,5,36246,263560,185709,68162,39690
2,6,45647,298522,218458,92722,47182
3,7,5884,305502,273477,108733,59787
4,8,36311,296193,225868,97921,50470
5,9,53051,289340,211300,78510,43843
6,10,34414,241853,168484,62436,3121


In [58]:
ActiveUser_trans=ActiveUser.T  # 행,렬 바꾸기

In [59]:
ActiveUser.T

,0,1,2,3,4,5,6
month,4,5,6,7,8,9,10
noActive,37042,36246,45647,5884,36311,53051,34414
some,283525,263560,298522,305502,296193,289340,241853
Active_low,197346,185709,218458,273477,225868,211300,168484
Active_med,70285,68162,92722,108733,97921,78510,62436
Active_high,39779,39690,47182,59787,50470,43843,3121


### Data-Preprocessing

In [60]:
#4월 한주 데이터
data4 = ActiveUser.T.iloc[1:,:1]
data4= data4.reset_index()
data4= data4.rename(columns={'index':'type',0 : 'value'})
data4['angle'] = data4['value']/data4['value'].sum() * 2*pi
data4['color'] = ["coral", "lightgreen", "lightblue", "orange", "navy"]
data4['percentage'] = data4['value']/data4['value'].sum()
#data4['percentage'] =  round((data4['value']/data4['value'].sum())*100,2)
data4

,type,value,angle,color,percentage
0,noActive,37042,0.370621,coral,0.058986
1,some,283525,2.836792,lightgreen,0.451489
2,Active_low,197346,1.974533,lightblue,0.314257
3,Active_med,70285,0.703232,orange,0.111923
4,Active_high,39779,0.398006,navy,0.063345


In [61]:
#5월 한주 데이터
data5 = ActiveUser.T.iloc[1:,1:2]
data5= data5.reset_index()
data5= data5.rename(columns={'index':'type',1 : 'value'})
data5['angle'] = data5['value']/data5['value'].sum() * 2*pi
data5['color'] = ["coral", "lightgreen", "lightblue", "orange", "navy"]
data5['percentage'] = data5['value']/data5['value'].sum()

#6월 한주 데이터
data6 = ActiveUser.T.iloc[1:,2:3]
data6= data6.reset_index()
data6= data6.rename(columns={'index':'type',2 : 'value'})
data6['angle'] = data5['value']/data5['value'].sum() * 2*pi
data6['color'] = ["coral", "lightgreen", "lightblue", "orange", "navy"]
data6['percentage'] = data6['value']/data6['value'].sum()

#7월 한주 데이터
data7 = ActiveUser.T.iloc[1:,3:4]
data7= data7.reset_index()
data7= data7.rename(columns={'index':'type',3 : 'value'})
data7['angle'] = data7['value']/data7['value'].sum() * 2*pi
data7['color'] = ["coral", "lightgreen", "lightblue", "orange", "navy"]
data7['percentage'] = data7['value']/data7['value'].sum()

## Pie-chart

In [62]:
output_file("pie.html")

### figure 안에 tooltips로 넣어도 되고, 따로 HoverTool변수에 값 넣고, figure에 tools안에 넣어도 됨.
# p = figure(plot_height=350, title="UserType (date:April)", toolbar_location=None,
#            tools="hover", tooltips="@type: @value", x_range=(-0.5, 1.0))
hover = HoverTool(tooltips=[("value","@value"),("percentage", "@percentage{%0.2f}")])

p = figure(plot_height=350, title="UserType (date:April)", toolbar_location=None,
           tools=["save",hover],  x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='type', source=data4)

#### 그래프 위에 라벨링 (LabelSet 쓰려면 ColumnDataSource 형태로 만들어야 함.)
data4["value"] = data4['value'].astype(str)
data4["value"] = data4["value"].str.pad(35, side = "left")
source = ColumnDataSource(data4)

labels = LabelSet(x=0, y=1, text='value', level='glyph', 
          angle=cumsum('angle', include_zero=True), source=source, render_mode='canvas')

p.add_layout(labels)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

#p.add_tools(HoverTool(tooltips=[("type","@type"),("value", "@value"), ("percentage", "@percentage{%0.2f}")]))
show(p)

## Multiple Pie-chart

In [63]:
####### multiple plot
from bokeh.layouts import gridplot
from bokeh.palettes import Viridis3
output_file("layout_grid.html")

# create three plots
p1 = figure(plot_height=400, title="UserType (date:April)", toolbar_location=None,
           tools=["save",hover],  x_range=(-0.5, 1.0))
p1.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='type', source=data4)

p2 = figure(plot_height=400, title="UserType (date:May)", toolbar_location=None,
           tools=["save",hover],  x_range=(-0.5, 1.0))
p2.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='type', source=data5)

p3 = figure(plot_height=400, title="UserType (date:June)", toolbar_location=None,
           tools=["save",hover],  x_range=(-0.5, 1.0))
p3.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='type', source=data6)

p4 = figure(plot_height=400, title="UserType (date:July)", toolbar_location=None,
           tools=["save",hover],  x_range=(-0.5, 1.0))
p4.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='type', source=data7)

# make a grid
grid = gridplot([[p1, p2], [p3, p4]])

# show the results
show(grid)

## Bar-chart

In [64]:
# 컬럼명 변경
ActiveUser_trans=ActiveUser_trans.rename(columns={0:"April",1:"May",2:"June",3:"July",4:"Aug",5:"Sep",6:"Oct"})
ActiveUser_trans=ActiveUser_trans.drop(ActiveUser_trans.index[0])  # 첫번째 month행 drop
ActiveUser_trans

,April,May,June,July,Aug,Sep,Oct
noActive,37042,36246,45647,5884,36311,53051,34414
some,283525,263560,298522,305502,296193,289340,241853
Active_low,197346,185709,218458,273477,225868,211300,168484
Active_med,70285,68162,92722,108733,97921,78510,62436
Active_high,39779,39690,47182,59787,50470,43843,3121


In [65]:
# Here is a list of categorical values (or factors)
types = ["noActive","some","Active_low","Active_med","Active_high"]

# Set the x_range to the list of categories above
p = figure(x_range=types, plot_height=250, title="Active User : April")

# Categorical values can also be used as coordinates
p.vbar(x=types, top=list(ActiveUser_trans.April), width=0.9)

############## 이부분이 interactive tool tip
hover = HoverTool(tooltips=[('User count', '@top')],
                          mode='vline')

p.add_tools(hover)
##############

# Set some properties to make the plot look better
p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

In [66]:
ActiveUser=ActiveUser.replace([4,5,6,7,8,9,10],['April','May','June','July','Aug','Sep','Oct'])
ActiveUser

,month,noActive,some,Active_low,Active_med,Active_high
0,April,37042,283525,197346,70285,39779
1,May,36246,263560,185709,68162,39690
2,June,45647,298522,218458,92722,47182
3,July,5884,305502,273477,108733,59787
4,Aug,36311,296193,225868,97921,50470
5,Sep,53051,289340,211300,78510,43843
6,Oct,34414,241853,168484,62436,3121


--------------------------------------------------
## groupby 한 데이터를 활용한 bar-chart
* grouping한 결과를 확인하려면 group.mean() 과 같이 집계함수가 함께 있어야 함.
* group의 month값이 4,5,6,7,8,9,10으로 숫자일 경우 x_range에 넣으면 에러남. 
    * df.month = df.month.astype(str) 이와같이 string타입으로 변경해주던지, 아니면 month값을 April,May..으로 바꾸던지 해야됨.
* index_cmap 변수는 컬러 설정을 위해 만든 변수

* groupby한 데이터로 Bokeh plot 그릴 경우 특이한점
    * x='usertype_month' 이렇게 언더바'_'로 표기하면 알아서 x축에 group대분류 안에 소분류가 나타남
    * value_mean 이경우도 알아서 value컬럼의 mean() 값으로 표현됨
* 컬러 파레트 설정할 때, factor_cmap(start=1, end=2) start값과 end값이 모두 있어야 정상적으로 채색됨.


### *melt , groupby*
* 사용자 유형에 따른 월별 수치를 보기 위해 melt로 month컬럼을 내리고
    * groupby 로 usertype 별, month별 수치 확인

In [67]:
df = pd.melt(ActiveUser, id_vars=['month'], var_name='usertype')
df.head(20)

,month,usertype,value
0,April,noActive,37042
1,May,noActive,36246
2,June,noActive,45647
3,July,noActive,5884
4,Aug,noActive,36311
5,Sep,noActive,53051
6,Oct,noActive,34414
7,April,some,283525
8,May,some,263560
9,June,some,298522


In [68]:
# melt와 groupby 사용해서 데이터 탐색
df = pd.melt(ActiveUser, id_vars=['month'], var_name='usertype')
group = df.groupby(by=['usertype','month'], sort=False).mean()    # month의 정렬을 a,b,c 순이 아닌 기존 정렬 유지하기 위해 sort=False지정
group.head(20)

value
usertype   month        
noActive   April   37042
           May     36246
           June    45647
           July     5884
           Aug     36311
           Sep     53051
           Oct     34414
some       April  283525
           May    263560
           June   298522
           July   305502
           Aug    296193
           Sep    289340
           Oct    241853
Active_low April  197346
           May    185709
           June   218458
           July   273477
           Aug    225868
           Sep    211300

In [69]:
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.palettes import Accent
from bokeh.palettes import Viridis
from bokeh.transform import factor_cmap

output_file("bar_pandas_groupby_nested.html")
ActiveUser=ActiveUser.replace([4,5,6,7,8,9,10],['April','May','June','July','Aug','Sep','Oct'])
df = pd.melt(ActiveUser, id_vars=['month'], var_name='usertype')
# figure 안에 x_range에 넣어주려면 month값은 string이어야 함.
#df.month = df.month.astype(str)

group = df.groupby(by=['usertype','month'], sort=False)

index_cmap = factor_cmap('usertype_month', palette=Viridis[7], factors=df.month, start=1, end=2)   # 이때 start, end 모두 설정해야 됨.

# source = ColumnDataSource(data=dict(group=group, ))
p= figure(plot_width=1300, plot_height=500, title="Active User", x_range=group, tooltips=[("type","@usertype_month"),("Counts","@value_mean")])
p.vbar(x='usertype_month', top='value_mean', width=1, source=group, line_color="white", fill_color=index_cmap)

p.y_range.start = 0
p.x_range.range_padding = 0.05
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Count by usertype"
p.xaxis.major_label_orientation = 1.2
p.outline_line_color = None

show(p)

------------------------------------
### dictionary형태의 데이터를 활용한 bar chart
* dictionary 형태의 data 포맷을 만들어 줌
* x축에 대분류안에 소분류를 나타내기 위해 types별로 months 값이 할당되도록 x변수 만들어 줌
* counts 변수에는 월별 데이터를 사용자 유형별 데이터순으로 나열
    * zip()함수 사용 => zip은 동일한 개수로 이루어진 자료형을 묶어주는 역할 
* ColumnDataSource는 Bokeh의 데이터 구조로 대부분의 plot, data tables 등이 ColumnDataSource로부터 나온다.
    * dictionary형태의 데이터를 넣어줌
* Hover는 figure안에 tooltips로 넣어도 되고, p.add_tools(HoverTool(tooltips=[()])) 로 넣어도 됨.

In [79]:
types = ["noActive","some","Active_low","Active_med","Active_high"]
months = ["April","May","June","July","Aug","Sep","Oct"]
data = {'types': types,
       'April': list(ActiveUser_trans['April']),
       'May' : list(ActiveUser_trans['May']),
       'June': list(ActiveUser_trans['June']),
       'July': list(ActiveUser_trans['July']),
       'Aug' :list(ActiveUser_trans['Aug']),
       'Sep':list(ActiveUser_trans['Sep']),
       'Oct':list(ActiveUser_trans['Oct'])}

In [80]:
data 

{'types': ['noActive', 'some', 'Active_low', 'Active_med', 'Active_high'],
 'April': [37042, 283525, 197346, 70285, 39779],
 'May': [36246, 263560, 185709, 68162, 39690],
 'June': [45647, 298522, 218458, 92722, 47182],
 'July': [5884, 305502, 273477, 108733, 59787],
 'Aug': [36311, 296193, 225868, 97921, 50470],
 'Sep': [53051, 289340, 211300, 78510, 43843],
 'Oct': [34414, 241853, 168484, 62436, 3121]}

In [82]:
x = [ (type, month) for type in types for month in months ]
x[:20]

[('noActive', 'April'),
 ('noActive', 'May'),
 ('noActive', 'June'),
 ('noActive', 'July'),
 ('noActive', 'Aug'),
 ('noActive', 'Sep'),
 ('noActive', 'Oct'),
 ('some', 'April'),
 ('some', 'May'),
 ('some', 'June'),
 ('some', 'July'),
 ('some', 'Aug'),
 ('some', 'Sep'),
 ('some', 'Oct'),
 ('Active_low', 'April'),
 ('Active_low', 'May'),
 ('Active_low', 'June'),
 ('Active_low', 'July'),
 ('Active_low', 'Aug'),
 ('Active_low', 'Sep')]

In [83]:
list(zip(data['April'], data['May'],data['June'],data['July'],data['Aug'],data['Sep'],data['Oct']))

[(37042, 36246, 45647, 5884, 36311, 53051, 34414),
 (283525, 263560, 298522, 305502, 296193, 289340, 241853),
 (197346, 185709, 218458, 273477, 225868, 211300, 168484),
 (70285, 68162, 92722, 108733, 97921, 78510, 62436),
 (39779, 39690, 47182, 59787, 50470, 43843, 3121)]

In [84]:
from bokeh.models import FactorRange
from bokeh.core.properties import value
from bokeh.palettes import Set3
# Set3[7]
# ['#8dd3c7', '#ffffb3', '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69']
types = ["noActive","some","Active_low","Active_med","Active_high"]
months = ["April","May","June","July","Aug","Sep","Oct"]
data = {'types': types,
       'April': list(ActiveUser_trans['April']),
       'May' : list(ActiveUser_trans['May']),
       'June': list(ActiveUser_trans['June']),
       'July': list(ActiveUser_trans['July']),
       'Aug' :list(ActiveUser_trans['Aug']),
       'Sep':list(ActiveUser_trans['Sep']),
       'Oct':list(ActiveUser_trans['Oct'])}

x = [ (type, month) for type in types for month in months ]
counts = sum(zip(data['April'], data['May'],data['June'],data['July'],data['Aug'],data['Sep'],data['Oct']), ())
### 위 counts와 아래 동일
# sum(zip(ActiveUser_trans['April'],ActiveUser_trans['May'],ActiveUser_trans['June'],ActiveUser_trans['July'],ActiveUser_trans['Aug'],ActiveUser_trans['Sep'],ActiveUser_trans['Oct']), ())

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x),plot_width =1300,plot_height=500, title="User Counts by month", tools="hover", tooltips=[("Type,Month","@x"),("User Count","@counts")])
p.vbar(x='x', top='counts', width=1, source=source,line_color="white",
       fill_color=factor_cmap('x', palette=Set3[7], factors=months,  start=1,end=2),
      )
#p.add_tools(HoverTool(tooltips=[("UserCount", "@counts")]))

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

----------------------------
## stack plot

In [78]:
from bokeh.palettes import Set3
from bokeh.palettes import Accent
# Accent[5]
# ['#7fc97f', '#beaed4', '#fdc086', '#ffff99', '#386cb0']
output_file("stacked.html")

months = ["April","May","June","July","Aug","Sep","Oct"]
types = ["noActive","some","Active_low","Active_med","Active_high"]
colors = ['#7fc97f', '#beaed4', '#fdc086', '#ffff99', '#386cb0']

data = {'months' : months,
        'noActive'   : list(ActiveUser['noActive']),
        'some'   : list(ActiveUser['some']),
        'Active_low'   : list(ActiveUser['Active_low']),
        'Active_med' : list(ActiveUser['Active_med']),
        'Active_high':list(ActiveUser['Active_high'])}

p = figure(x_range=months,plot_width =850,plot_height=650, title="Ratio_stack plot",
           toolbar_location=None, tools="hover", tooltips="$name: @$name")

p.vbar_stack(types, x='months', width=0.9, color = colors,source=data, legend=[value(x) for x in types])

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)